In [8]:
import os
import pandas as pd
from datetime import datetime
import urllib
import timeit
import numpy as np


In [1]:
def fileParser(fileInput,tempName,uncertaintyName):
    f = open(fileInput,'r')
    lines = f.readlines()
    f.close()
    
    colYear = []
    colMonth = []
    colTempName = []
    colUncertaintyName = []
    
    count = 0;
    for line in lines:
        if not line.startswith('%'):
            line_list = line.split()
            line_list = [float(i) for i in line_list]
            if len(line_list):
                count += 1
                colYear.append(line_list[0])
                colMonth.append(line_list[1])
                colTempName.append(line_list[2])
                colUncertaintyName.append(line_list[3])
                
                               
    dfTemp = pd.DataFrame(data=np.transpose([colYear,colMonth,colTempName,colUncertaintyName]),columns=['Year','Month',tempName,uncertaintyName])

    dfTemp['dt'] = None
    for row in dfTemp.index:
        dfTemp.iloc[row,4] = datetime(int(dfTemp.iloc[row, 0]),int(dfTemp.iloc[row, 1]),1)
    dfTemp = dfTemp.set_index(dfTemp['dt'])
    del(dfTemp['Year'])
    del(dfTemp['Month'])
    del(dfTemp['dt'])
    
    return dfTemp

In [3]:
def addAbsolute(dfAnomaly,listAbsoluteTemp):
    for i in range(1, 13):
        for row in dfAnomaly[dfAnomaly.index.month ==i].index:
            dfAnomaly.ix[row,0] =  dfAnomaly.ix[row,0] + float(listAbsoluteTemp[i-1]) 
    return dfAnomaly


In [4]:
def findAbsoluteTemp(fileInput):
    f = open(fileInput, 'r')
    lines = f.readlines()
    f.close()
    key = []
    
    for idx, line in enumerate(lines):
        if ''.join(line.split()).startswith('%JanFebMarAprMayJunJulAugSepOctNovDec'):
            key = lines[idx + 1].replace('%','').split()
            
    return key

In [5]:
def getCountryFromFileName(fileName, delimiter):
    return ' '.join(fileName.split('-')[0:fileName.split('-').index(delimiter)]).title()

In [6]:
def getPlaceFromFileName(fileName, delimiter, hasLatLong=False):
    raw = ' '.join(fileName.split('-')[0:fileName.split('-').index(delimiter)]).title().split('%3A%3A')
    filtered = list(map(lambda x: urllib.parse.unquote(x, encoding='latin-1').title(), raw))
    if hasLatLong:
        coords = filtered[-1].split(' ')
        filtered[-1:] = coords
    return filtered

# Global Average, Max & Min For Land and Ocean and Land 

In [71]:
fileInput = './Complete_TAVG_complete.txt'

dfGlobalLandAvg = fileParser(fileInput,'LandAverageTemperature','LandAverageTemperatureUncertainty')
TAVG_KEY = findAbsoluteTemp(fileInput)
dfGlobalLandAvgClean= addAbsolute(dfGlobalLandAvg,TAVG_KEY)


In [72]:
fileInput = './Complete_TMAX_complete.txt'

dfGlobalLandMax = fileParser(fileInput,'LandMaxTemperature','LandMaxTemperatureUncertainty')


TMAX_KEY =  findAbsoluteTemp(fileInput)
dfGlobalLandMaxClean= addAbsolute(dfGlobalLandMax,TMAX_KEY)

In [73]:
fileInput = './Complete_TMIN_complete.txt'

dfGlobalLandMin = fileParser(fileInput,'LandMinTemperature','LandMinTemperatureUncertainty')

TMIN_KEY =  findAbsoluteTemp(fileInput)
dfGlobalLandMinClean= addAbsolute(dfGlobalLandMin,TMIN_KEY)

In [89]:
fileInput = './Land_and_Ocean_complete.txt'

dfGlobalLandAndOceanAvg = fileParser(fileInput,'LandAndOceanAverageTemperature','LandAndOceanAverageTemperatureUncertainty')
dfGlobalLandAndOceanAvg = dfGlobalLandAndOceanAvg[int(len(dfGlobalLandAndOceanAvg)/2):] #there are two sets of readings stacked in this data set

T_KEY =  findAbsoluteTemp(fileInput)
dfGlobalLandAndOceanAvg = addAbsolute(dfGlobalLandAndOceanAvg,T_KEY)



In [92]:
dfGlobal = dfGlobalLandAvgClean.join(dfGlobalLandMaxClean).join(dfGlobalLandMinClean).join(dfGlobalLandAndOceanAvg)
dfGlobal.to_csv("GlobalTemperatures.csv")

# By Country

In [105]:
count = 0 

for fn in os.listdir('./country/'):
    if os.path.isfile('./country/' + fn):
            count += 1
            fileInput = './country/' + fn
            fn = urllib.parse.unquote(fn, encoding='latin-1') #to manage the file encoding issie
            country = getCountryFromFileName(fn,'TAVG')
            print(country)
            dfTemp = fileParser(fileInput,'AverageTemperature','AverageTemperatureUncertainty')
            T_KEY =  findAbsoluteTemp(fileInput)
            dfTemp = addAbsolute(dfTemp,T_KEY)
            dfTemp['Country'] = country
            if count == 1:
                dfGlobalLandByCountry = dfTemp
            else:
                dfGlobalLandByCountry = dfGlobalLandByCountry.append(dfTemp)
        
                


Åland
Afghanistan
Africa
Albania
Algeria
American Samoa
Andorra
Angola
Anguilla
Antarctica
Antigua And Barbuda
Argentina
Armenia
Aruba
Asia
Australia
Austria
Azerbaijan
Bahamas
Bahrain
Baker Island
Bangladesh
Barbados
Belarus
Belgium
Belize
Benin
Bhutan
Bolivia
Bonaire, Saint Eustatius And Saba
Bosnia And Herzegovina
Botswana
Brazil
British Virgin Islands
Bulgaria
Burkina Faso
Burma
Burundi
Côte D'Ivoire
Cambodia
Cameroon
Canada
Cape Verde
Cayman Islands
Central African Republic
Chad
Chile
China
Christmas Island
Colombia
Comoros
Congo (Democratic Republic Of The)
Congo
Costa Rica
Croatia
Cuba
Curaçao
Cyprus
Czech Republic
Denmark (Europe)
Denmark
Djibouti
Dominica
Dominican Republic
Ecuador
Egypt
El Salvador
Equatorial Guinea
Eritrea
Estonia
Ethiopia
Europe
Falkland Islands (Islas Malvinas)
Faroe Islands
Federated States Of Micronesia
Fiji
Finland
France (Europe)
France
French Guiana
French Polynesia
French Southern And Antarctic Lands
Gabon
Gambia
Gaza Strip
Georgia
Germany
Ghana
Gree

In [106]:
dfGlobalLandByCountry.to_csv('GlobalLandTemperaturesByCountry.csv')

# By State

In [9]:
count = 0 

for fn in os.listdir('./state/'):
    if os.path.isfile('./state/' + fn):
            start_time = timeit.default_timer()
            count += 1
            fileInput = './state/' + fn
            place = getPlaceFromFileName(fn,'TAVG')
            print(place)
            #print('time point 1 ' + str(timeit.default_timer() - start_time))
            dfTemp = fileParser(fileInput,'AverageTemperature','AverageTemperatureUncertainty')
            #print('time point 2 ' + str(timeit.default_timer() - start_time))
            T_KEY =  findAbsoluteTemp(fileInput)
            dfTemp = addAbsolute(dfTemp,T_KEY)
            #print('time point 3 ' + str(timeit.default_timer() - start_time))
            
            dfTemp['State'] = place[0]
            dfTemp['Country'] = place[1]
            
            if count == 1:
                dfGlobalLandByState = dfTemp
            else:
                dfGlobalLandByState = dfGlobalLandByState.append(dfTemp)
            
            print('time point 4 ' + str(timeit.default_timer() - start_time))
                

['Acre', 'Brazil']
time point 4 2.1099909205747664
['Adygey', 'Russia']
time point 4 3.5751113216485066
['Aga Buryat', 'Russia']
time point 4 2.4775394757618834
['Alabama', 'United States']
time point 4 3.5841946944576044
['Alagoas', 'Brazil']
time point 4 2.330435417653561
['Alaska', 'United States']
time point 4 2.3713469129954206
['Alberta', 'Canada']
time point 4 3.2088532291173237
['Altay', 'Russia']
time point 4 2.481267171956418
['Amazonas', 'Brazil']
time point 4 2.207736459813674
['Amur', 'Russia']
time point 4 2.599651429022579
['Andaman And Nicobar', 'India']
time point 4 2.8418364124427598
['Andhra Pradesh', 'India']
time point 4 2.892689483657037
['Anhui', 'China']
time point 4 2.2551251381651696
['Arizona', 'United States']
time point 4 2.289974735512395
['Arkansas', 'United States']
time point 4 3.4100998736775594
["Arkhangel'Sk", 'Russia']
time point 4 3.627920022106423
['Arunachal Pradesh', 'India']
time point 4 2.5893991789041593
['Assam', 'India']


KeyboardInterrupt: 

In [10]:
dfGlobalLandByState.to_csv('GlobalLandTemperaturesByState.csv')

# Major Cities

In [119]:
count = 0 

for fn in os.listdir('./city_major/'):
    if os.path.isfile('./city_major/' + fn):
            start_time = timeit.default_timer()
            count += 1
            fileInput = './city_major/' + fn
            place = getPlaceFromFileName(fn,'TAVG',True)
            print(place)
            #print('time point 1 ' + str(timeit.default_timer() - start_time))
            dfTemp = fileParser(fileInput,'AverageTemperature','AverageTemperatureUncertainty')
            #print('time point 2 ' + str(timeit.default_timer() - start_time))
            T_KEY =  findAbsoluteTemp(fileInput)
            dfTemp = addAbsolute(dfTemp,T_KEY)
            #print('time point 3 ' + str(timeit.default_timer() - start_time))
            
            dfTemp['City'] = place[0]
            dfTemp['Country'] = place[1]
            dfTemp['Latitude'] = place[2]
            dfTemp['Longitude'] = place[3]

            
            if count == 1:
                dfGlobalLandByMajorCity = dfTemp
            else:
                dfGlobalLandByMajorCity = dfGlobalLandByMajorCity.append(dfTemp)
            
            print('time point 4 ' + str(timeit.default_timer() - start_time))
                

['Abidjan', "Côte D'Ivoire", '5.63N', '3.23W']
time point 4 8.350980439372506
['Addis Abeba', 'Ethiopia', '8.84N', '38.11E']
time point 4 3.7978115435762447
['Ahmadabad', 'India', '23.31N', '72.52E']
time point 4 9.578717598800722
['Aleppo', 'Syria', '36.17N', '37.79E']
time point 4 9.468627637523241
['Alexandria', 'Egypt', '31.35N', '30.16E']
time point 4 5.045611066789206
['Ankara', 'Turkey', '39.38N', '33.29E']
time point 4 10.172836265262958
['Baghdad', 'Iraq', '32.95N', '45.00E']
time point 4 8.733702536830606
['Bangalore', 'India', '12.05N', '77.26E']
time point 4 8.446379501665433
['Bangkok', 'Thailand', '13.66N', '99.91E']
time point 4 4.471069607188838
['Belo Horizonte', 'Brazil', '20.09S', '44.36W']
time point 4 6.713837946535932
['Berlin', 'Germany', '52.24N', '13.14E']
time point 4 6.49520468371702
['Bogotá', 'Colombia', '4.02N', '74.73W']
time point 4 8.120483530521597
['Bombay', 'India', '18.48N', '72.68E']
time point 4 4.350569891610576
['Brasília', 'Brazil', '15.27S', '

In [120]:
dfGlobalLandByMajorCity.to_csv('GlobalLandTemperaturesByMajorCity.csv')

# All Cities

In [13]:
count = 0 

for fn in os.listdir('./city/'):
    if os.path.isfile('./city/' + fn):
            start_time = timeit.default_timer()
            count += 1
            fileInput = './city/' + fn
            place = getPlaceFromFileName(fn,'TAVG',True)
            print(place)
            #print('time point 1 ' + str(timeit.default_timer() - start_time))
            dfTemp = fileParser(fileInput,'AverageTemperature','AverageTemperatureUncertainty')
            #print('time point 2 ' + str(timeit.default_timer() - start_time))
            T_KEY =  findAbsoluteTemp(fileInput)
            dfTemp = addAbsolute(dfTemp,T_KEY)
            #print('time point 3 ' + str(timeit.default_timer() - start_time))
            
            dfTemp['City'] = place[0]
            dfTemp['Country'] = place[1]
            dfTemp['Latitude'] = place[2]
            dfTemp['Longitude'] = place[3]

            
            if count == 1:
                dfGlobalLandByCity = dfTemp
            else:
                dfGlobalLandByCity = dfGlobalLandByCity.append(dfTemp)
            
            print('time point 4 ' + str(timeit.default_timer() - start_time))

['Århus', 'Denmark', '57.05N', '10.33E']
time point 4 4.896085583451793
['Çorlu', 'Turkey', '40.99N', '27.69E']
time point 4 5.267944102321167
['Çorum', 'Turkey', '40.99N', '34.08E']
time point 4 5.409550371072157
['Öskemen', 'Kazakhstan', '50.63N', '82.39E']
time point 4 4.102293541765334
['Ürümqi', 'China', '44.20N', '87.20E']
time point 4 3.955216327175094
['A Coruña', 'Spain', '42.59N', '8.73W']
time point 4 5.806014527080379
['Aachen', 'Germany', '50.63N', '6.34E']
time point 4 5.8559501815885255
['Aalborg', 'Denmark', '57.05N', '10.33E']
time point 4 5.635417653560751
['Aba', 'Nigeria', '5.63N', '8.07E']
time point 4 3.465778856781924
['Abadan', 'Iran', '29.74N', '48.00E']
time point 4 3.597380388441536
['Abakaliki', 'Nigeria', '5.63N', '8.07E']
time point 4 3.3759588662560986
['Abakan', 'Russia', '53.84N', '91.36E']
time point 4 4.229015474498681
['Abbotsford', 'Canada', '49.03N', '122.45W']
time point 4 4.035231722722244
['Abengourou', "Côte D'Ivoire", '7.23N', '4.05W']
time po

In [14]:
dfGlobalLandByCity.to_csv('GlobalLandTemperaturesByCity.csv')